## bert-as-service

bert-as-service库是另一个广泛使用的BERT库。它很简单，可扩展，且易用

### 安装bert-as-service库

In [ ]:
! pip install tensorflow==1.14
! pip install bert-serving-client
! pip install -U bert-serving-server[http]

### 计算句子特征

首先，下载并解压要使用的预训练BERT模型。在这个例子中，我们使用预训练的BERT-base-uncased模型

In [ ]:
! wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

In [ ]:
! unzip uncased_L-12_H-768_A-12.zip

然后，我们启动BERT模型服务器。在启动BERT模型服务器的同时，需要设置要用到的汇聚方法。也就是说，BERT模型返回句子中每个单词的特征，然后通过汇聚方法得到整个句子的特征。在这个例子中，我们使用**平均汇聚法**

In [ ]:
!nohup bert-serving-start -pooling_strategy REDUCE_MEAN -model_dir=./uncased_L-12_H-768_A-12 > out.file 2>&1 &

In [ ]:
from bert_serving.client import BertClient

In [ ]:
bc = BertClient()

In [ ]:
sentence1 = 'the weather is great today'
sentence2 = 'it looks like today the weather is pretty nice'

In [ ]:
sent_rep1 = bc.encode([sentence1])
sent_rep2 = bc.encode([sentence2])

In [ ]:
print(sent_rep1.shape, sent_rep2.shape)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(sent_rep1,sent_rep2)

### 计算上下文的单词特征

BERT模型返回句子中每个单词的特征，而一个单词的特征基于句中单词的上下文。为了获得单词的特征，在启动BERT模型服务器时将汇聚方法设置为`NONE`

In [ ]:
!nohup bert-serving-start -max_seq_len=20 -pooling_strategy NONE -model_dir=./uncased_L-12_H-768_A-12 > out.file 2>&1 &

In [ ]:
from bert_serving.client import BertClient
bc = BertClient()

In [ ]:
sentence = 'The weather is great today'

In [ ]:
vec = bc.encode([sentence])

In [ ]:
print(vec.shape)

- vec[0][0]：保存[CLS]标记的特征。
- vec[0][1]：保存句子中第1个词的特征"the"。
- vec[0][2]：保存句子中第2个词的特征"weather"。
- vec[0][3]：保存句子中第3三个词的特征"is"。
- vec[0][4]：保存句子中第4个词的特征"great"。
- vec[0][5]：保存句子中第5个词的特征"today"。
- vec[0][6]：保存[SEP]标记的特征。
- vec[0][7]至vec[0][20]：保存填充标记的特征。